In [1]:
import http.client
import json
import urllib.parse
import re
import pandas as pd
from shapely.geometry import Point, Polygon, MultiPolygon
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
import os

In [2]:
# Function to parse the curl bash command into usable components
def clean_curl_request(curl_command):
    # Extract the URL
    url_pattern = r"curl '(.*?)'"
    match = re.search(url_pattern, curl_command)
    if match:
        url = match.group(1)
    else:
        raise ValueError("URL not found in the curl command")

    # Extract headers
    headers = {}
    header_pattern = r"-H '([^:]+): (.*?)'"
    for header_match in re.finditer(header_pattern, curl_command):
        header_name = header_match.group(1)
        header_value = header_match.group(2)
        headers[header_name] = header_value

    # Extract data
    data_pattern = r"--data-raw '(.*?)'"
    match = re.search(data_pattern, curl_command)
    data = {}
    if match:
        data_string = match.group(1)
        data_items = data_string.split("&")
        for item in data_items:
            key, value = item.split("=")
            data[key] = urllib.parse.unquote(value)

    # Parse URL for host and path
    parsed_url = urllib.parse.urlparse(url)
    host = parsed_url.netloc
    path = parsed_url.path + ("?" + parsed_url.query if parsed_url.query else "")

    return host, path, data, headers


In [3]:
def send_request(session, url, data, headers, lat, lng):
    # Update latitude and longitude in the data dictionary
    data["lat"] = str(lat)
    data["lng"] = str(lng)
    
    # Measure request time
    start_time = time.time()
    
    response = session.post(url, data=data, headers=headers)
    elapsed_time = time.time() - start_time  # Calculate elapsed time
    
    # Print the response time
    print(f"Response received in {elapsed_time:.2f} seconds for lat: {lat}, lng: {lng}")
    
    # Check for a successful response and return the result
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None


In [4]:
import json
import numpy as np
import pandas as pd
from shapely.geometry import Polygon, Point

def load_geojson(filepath, ilce):
    with open(filepath, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return [feature for feature in data["features"] if feature["properties"].get("name") == ilce]

def extract_polygons(features):
    polygons = []
    for feature in features:
        geometry_type = feature["geometry"].get("type")
        coordinates = feature["geometry"].get("coordinates")
        if geometry_type == "Polygon":
            polygons.extend([Polygon(coordinates[0])])
        elif geometry_type == "MultiPolygon":
            polygons.extend([Polygon(polygon[0]) for polygon in coordinates])
        else:
            raise ValueError(f"Unsupported geometry type: {geometry_type}")
    return polygons

def generate_grid(polygons, aralik):
    red_points = []
    lat_spacing = aralik / 111320
    for polygon in polygons:
        min_lat = min(polygon.exterior.coords.xy[1])
        max_lat = max(polygon.exterior.coords.xy[1])
        lat_lines = np.arange(min_lat, max_lat, lat_spacing)
        for lat in lat_lines:
            min_lon = min(polygon.exterior.coords.xy[0])
            max_lon = max(polygon.exterior.coords.xy[0])
            lon_spacing = aralik / (111320 * np.cos(np.radians(lat)))
            lon_points = np.arange(min_lon, max_lon, lon_spacing)
            for lon in lon_points:
                point = Point(lon, lat)
                if any(polygon.contains(point) for polygon in polygons):
                    red_points.append((lon, lat))
    return red_points,polygons

def process_api_requests(red_points, send_request_func, polygons):
    columns = ["tesis_adi", "il_adi", "sokak_adi", "ilce_adi", "mahalle_adi", "lng", "lat", "id", "polygon_bounds"]
    df = pd.DataFrame(columns=columns)
    for lng, lat in red_points:
        result = send_request_func(lat, lng)
        if isinstance(result, dict) and "features" in result:
            for feature in result["features"]:
                properties = feature["properties"]
                row = {
                    "tesis_adi": properties.get("tesis_adi", ""),
                    "il_adi": properties.get("il_adi", ""),
                    "sokak_adi": properties.get("sokak_adi", ""),
                    "ilce_adi": properties.get("ilce_adi", ""),
                    "mahalle_adi": properties.get("mahalle_adi", ""),
                    "lng": lng,
                    "lat": lat,
                    "id": properties.get("id", ""),
                    "polygon_bounds": polygons  # Now passed explicitly
                }
                df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    return df.drop_duplicates(subset=["id"])

def save_dataframe(df, filename):
    """
    Saves the given DataFrame to a CSV file.

    Args:
        df (pd.DataFrame): The DataFrame to save.
        filename (str): The name of the CSV file.
    """
    try:
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Data saved successfully to {filename}")
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")



def process_district(ilce, aralik, geojson_filepath, curl_command):
    # Parse the curl command
    host, path, data, headers = clean_curl_request(curl_command)
    url = f"https://{host}{path}"
    
    # Load GeoJSON and process data
    features = load_geojson(geojson_filepath, ilce)
    polygons = extract_polygons(features)
    red_points, polygons = generate_grid(polygons, aralik)  # Unpack both
    
    # Create a session and update headers
    session = requests.Session()
    session.headers.update(headers)
    
    # Define the function for API requests
    send_request_func = lambda lat, lng: send_request(session, url, data, headers, lat, lng)
    
    # Process API requests
    df = process_api_requests(red_points, send_request_func, polygons)  # Pass polygons explicitly
    print(df.head(10))
    
    # Save results
    save_dataframe(df, f"./results/{ilce}_results.csv")









In [5]:
# Example usage
curl_command = """
curl 'https://www.turkiye.gov.tr/afet-ve-acil-durum-yonetimi-acil-toplanma-alani-sorgulama?harita=goster&submit' \
  -H 'Accept: application/json, text/javascript, */*; q=0.01' \
  -H 'Accept-Language: tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7' \
  -H 'Connection: keep-alive' \
  -H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' \
  -H 'Cookie: ridbb=WyI1ZWZjYWJjN2E5YzNiY2M4ZDY4OWExMGIwN2IwMTcyMmNjZDc5NjUzZDBlOCJd; _uid=1721632846-0ba03683-f088-45be-ab36-f8dd67f8b32d; w3p=1943251136.20480.0000; uyari=b8e23373d48d32ab19013491277695691c051a6639e16343c50db3d9f71adf64; language=tr_TR.UTF-8; TURKIYESESSIONID=qdcfg9njnp3l6rpuap8h31sc89; TS015d3f68=015c1cbb6dbfba1ef7c0048b7c2ad6f0bd50242051ecac5efb846f3755bf12a28f27832a03e304de5d0e33cfbff305679c0ff8db00; _lastptts=1736729283' \
  -H 'Origin: https://www.turkiye.gov.tr' \
  -H 'Referer: https://www.turkiye.gov.tr/afet-ve-acil-durum-yonetimi-acil-toplanma-alani-sorgulama?harita=goster' \
  -H 'Sec-Fetch-Dest: empty' \
  -H 'Sec-Fetch-Mode: cors' \
  -H 'Sec-Fetch-Site: same-origin' \
  -H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 OPR/115.0.0.0' \
  -H 'X-Requested-With: XMLHttpRequest' \
  -H 'sec-ch-ua: "Chromium";v="130", "Opera";v="115", "Not?A_Brand";v="99"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Windows"' \
  --data-raw 'pn=%2Fafet-ve-acil-durum-yonetimi-acil-toplanma-alani-sorgulama&ajax=1&token=%7BCD55F0-D0134D-833049-114286-E8DAAA-ADC974-DA2C4F-3A1FF0%7D&islem=getAlanlarForNokta&lat=40.05452994008135&lng=39.90579584245058'
"""
# Replace `send_request` with your actual API call function.
#process_district("Silivri", 100, "istanbul-admin-level-6.geojson", curl_command)

In [6]:
with open('istanbul-admin-level-8-without-centers.geojson', 'r',encoding='utf-8') as file:
    data = json.load(file)

completed_neighborhoods = []

directory = os.fsencode('./results')
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"): 
        completed_neighborhoods.append((os.path.splitext(os.path.basename(filename))[0]).split('_',1)[0])
        continue
    else:
        continue

for i in data["features"]:
    if i["properties"].get("name", "point") == 'point':
        continue
    elif i["properties"].get("name", "point") in completed_neighborhoods:
        print(i["properties"].get("name", "point"),": in results")
        continue
    else:
        print(i["properties"].get("name", "point"))
        ilce = i["properties"].get("name", "point")
        process_district(ilce, 100, "istanbul-admin-level-8-without-centers.geojson", curl_command)
        completed_neighborhoods.append(ilce)

Kadıköy Mahallesi : in results
Çelebi Mahallesi : in results
Paşaköy Mahallesi : in results
Ayvansaray Mahallesi : in results
Balat Mahallesi : in results
Derviş Ali Mahallesi : in results
Atikali Mahallesi : in results
Yavuz Sultan Selim Mahallesi : in results
Karagümrük Mahallesi : in results
Hırka-i Şerif Mahallesi : in results
Topkapı Mahallesi : in results
İçerenköy Mahallesi : in results
Cumhuriyet Mahallesi : in results
Ünalan Mahallesi : in results
Bulgurlu Mahallesi : in results
Kısıklı Mahallesi : in results
Yunus Emre Mahallesi : in results
Atatürk Mahallesi : in results
Anadolu Mahallesi : in results
Hicret Mahallesi : in results
Bolluca Mahallesi : in results
Mavigöl Mahallesi : in results
Gümüşyaka Mahallesi : in results
Bahçelievler Mahallesi : in results
Siyavuşpaşa Mahallesi : in results
Şehremini Mahallesi : in results
Basınköy Mahallesi : in results
Mevlanakapı Mahallesi : in results
Seyyid Ömer Mahallesi : in results
Silivrikapı Mahallesi : in results
Sümbül Efendi 

In [9]:
for i in data["features"]:
    print(i["properties"].get("@id", "point") )

relation/1275322
relation/1275323
relation/1276035
relation/2463274
relation/2463371
relation/2463390
relation/2464768
relation/2464918
relation/2468218
relation/2468219
relation/2468242
relation/3403828
relation/3866327
relation/3866328
relation/3866467
relation/3866468
relation/6648205
relation/6648277
relation/6648717
relation/6648841
relation/6648854
relation/6648907
relation/6670488
relation/6841016
relation/6841110
relation/6868850
relation/7404512
relation/7648808
relation/7648809
relation/7648810
relation/7648811
relation/7648812
relation/7648813
relation/7648814
relation/7648815
relation/7648816
relation/7648817
relation/7648818
relation/7648819
relation/7648820
relation/7648821
relation/7648822
relation/7648823
relation/7648824
relation/7648825
relation/7648826
relation/7648827
relation/7648828
relation/7648829
relation/7648830
relation/7648831
relation/7648832
relation/7648833
relation/7648834
relation/7648835
relation/7648836
relation/7648837
relation/7648838
relation/76488

In [ ]:
import os

# Directory containing the files
directory = "./results"

# Loop through files in the directory
for filename in os.listdir(directory):
    # Construct the full file path
    old_file_path = os.path.join(directory, filename)
    
    # Skip directories
    if os.path.isdir(old_file_path):
        continue
    
    # Create a new name for the file
    new_name = f"new_prefix_{filename}"
    new_file_path = os.path.join(directory, new_name)
    
    # Rename the file
    os.rename(old_file_path, new_file_path)
    print(f"Renamed: {filename} -> {new_name}")
